# Loss Function Experiments


In [ ]:
%load_ext autoreload

import numpy as np
import random
import torch
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

from tqdm.notebook import tqdm

import open3d as o3d
import scipy.stats

# from src.elements import *
# from src.ifc import *
# from src.preparation import *
from src.visualisation import *
from src.chamfer import *
from src.morph import *

random.seed = 42


This notebook contains experiments on loss functions.

Specifically it contains;

1. Analysis of results from sphere morphing, including consistency metrics
2. Visualising point-wise distances from a loss function
3. Measuring loss correlation in results from completion models
4. Visualising loss curves for model training

#### data loading and pre-processing

All data from sphere morphing must be loaded for analysis. The morph notebook must be run prior to this section in order to generate results from sphere morphing.


In [ ]:
# helper code to recombine batched output from uniformCD
file_prefix = "data/uniform_metrics"
icd_chamfer_list, icd_emd_list, icd_assignment_list = [], [], []

for i in range(1,9):
    with open(file_prefix + str(i) + ".pkl", "rb") as f:
        chamfer, emd, ass = pickle.load(f)
        print(len(ass), len(ass[0]), len(ass[0][0]), ass[0][0][0].shape)
        icd_chamfer_list.append(chamfer)
        icd_emd_list.append(emd)
        icd_assignment_list.append(ass[0])

icd_assignment_list = np.array(icd_assignment_list)        
icd_assignment_list = np.transpose(icd_assignment_list, axes=(1,2,0,3,4))
icd_assignment_list = np.reshape(icd_assignment_list, (101, 2, 8*150, 4096))

In [ ]:
uniform_cd = np.sum(np.array(icd_chamfer_list), axis=0)
uniform_emd = np.sum(np.array(icd_emd_list), axis=0)
print(uniform_emd.shape)

In [ ]:
# load metrics
loss_funcs = ["emd"]
#loss_funcs = ["infocd", "chamfer", "emd", "uniform"]
chamfer_list, emd_list = [], []
for loss_func in loss_funcs:
    if loss_func == "uniform":
        continue
    with open("data/" + loss_func + "_metrics.pkl", "rb") as f:
        chamfer, emd, ass = pickle.load(f)
        chamfer_list.append(chamfer)
        emd_list.append(emd)


In [ ]:
if "uniform" in loss_funcs:
    chamfer_list.append(uniform_cd)
    emd_list.append(uniform_emd)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 5))
plt.rcParams.update({'font.size': 12})
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14) 
loss_funcs = ["infoCD", "CD", "EMD", "UniformCD"]

plot_dists(axes[1], chamfer_list, loss_funcs, "Chamfer Distance", "iterations", log=True)
plot_dists(axes[0], emd_list, loss_funcs, "Earth Mover's Distance", "iterations", log=True)
print("EMD infocd", emd_list[0][-1], "chamfer", emd_list[1][-1],  "emd", emd_list[2][-1], "uniform", emd_list[3][-1])
print("CD infocd", chamfer_list[0][-1], "chamfer", chamfer_list[1][-1], "emd", chamfer_list[2][-1], "uniform", chamfer_list[3][-1])


In [ ]:
print(len(ass), len(ass[0]), ass[0][0].shape, len(ass[0][0]), ass[0][0][0].shape)
print(emd_list[1][0])

In [ ]:
# load losses
# single, not batch
loss_types = ["reverse", "chamfer", "emd", "pair"]
losses = []

for loss_func in loss_types:
    with open("data/loss_" + loss_func + ".pkl", "rb") as f:
        losses.append(pickle.load(f))
        
plot_dists(losses, loss_types, "loss function comparison")


#### consistency metrics

We measure;

1. backward assignment consistency
2. coorrespondence coverage
3. correspondence stability


In [ ]:
# measure conssistency between forward and backward correspondences for chamfer distance
# optionally compare against the ideal assignment, as measured by EMD
def measure_assignment_consistency(assignment, emd=None):
    reverse_assignment = torch.gather(assignment[0], 0, assignment[1])
    expected = torch.arange(assignment[0].shape[0], device=torch.device("cuda"))
    consistency = torch.sum(torch.eq(expected, reverse_assignment).long())
    print("consistency", consistency.item(), len(torch.unique(assignment[0])), len(torch.unique(assignment[1])))
    
    if emd is not None:
        #print(emd[:5], assignment[0][:5], assignment[1][:5])
        emd_consistency = torch.sum(torch.eq(emd, assignment[0]).long())
        #print("emd_consistency", emd_consistency.item(), len(torch.unique(emd)))


In [ ]:
# measure conssistency for EMD approx correspondences
# this is different from other distances as only a one way assignment is returned
def measure_batch_assignment_consistency_emd(assignment):
    consistencies = np.array([4096 for i in range(assignment.shape[-1])])
    cuda = torch.device("cuda")

    uniques = []
    changes = []
    for i in tqdm(range(len(assignment))):
        class_assignment = torch.tensor(assignment[i], device=cuda)
        
        unique = 0
        for j in range(len(class_assignment)):
            unique += len(torch.unique(class_assignment[j]))
        unique = (unique/len(class_assignment))
        uniques.append(unique)
        
        # check rate of change of matches
        if i==0:
            change = 0
        else:
            change = (assignment[i] == assignment[i-1]).sum()
            change = (change/len(class_assignment[0]))
        changes.append(change)
        
        print("un", unique, "change", change)
        
    # save results
    with open("data/" + "emd" + "_consistency.pkl", "wb") as f:
        pickle.dump([consistencies, uniques, changes], f)

In [ ]:
measure_batch_assignment_consistency_emd(ass)

In [ ]:
# measure consistency between forward and backward correspondences
# measure number of unique assignments
# measure rate of change of assignments
def measure_batch_assignment_consistency(assignment, loss):
    cuda = torch.device("cuda")
    # loop through iterations
    consistencies = []
    uniques = []
    changes = []
    for i in tqdm(range(len(assignment))):
        
        # check reverse consistency
        class_assignment = torch.tensor(assignment[i], device=cuda)
        reverse_assignment = torch.gather(class_assignment[0], 1, class_assignment[1])
        #print("r", reverse_assignment.shape)
        expected = torch.arange(class_assignment[0].shape[1], device=cuda)
        expected = expected.repeat(class_assignment[0].shape[0], 1)
        #print("e", expected.shape)
        consistency = torch.sum(torch.eq(expected, reverse_assignment).long())/len(class_assignment[0])
        consistencies.append(consistency.item())
        
        # check uniqueness of matches
        unique = 0
        for j in range(len(class_assignment[0])):
            unique += (len(torch.unique(class_assignment[0][j])) +
                       len(torch.unique(class_assignment[1][j])))
        unique = (unique/len(class_assignment[0]))/2
        uniques.append(unique)
        
        # check rate of change of matches
        if i==0:
            change = 0
        else:
            change = (assignment[i] == assignment[i-1]).sum()
            change = (change/len(class_assignment[0]))/2
        changes.append(change)
        
        print("un", unique, "consistency", consistency.item(), "change", change)

    
    # save results
    with open("data/" + loss + "_consistency.pkl", "wb") as f:
        pickle.dump([consistencies, uniques, changes], f)


In [ ]:
# preprocess
loss = "emd"
#ass = np.array(ass)

if loss == "infocd":
    ass = np.transpose(ass, axes=(1,2,0,3,4,5))
    ass = np.reshape(ass, (101, 2, 8*150, 4096))
    
if loss == "chamfer":
    ass = np.transpose(ass, axes=(1,2,0,3,4))
    ass = np.reshape(ass, (101, 2, 8*150, 4096))
    
if loss == "emd":
    ass = np.array(ass)
    ass = np.transpose(ass, axes=(1,0,2,3))
    ass = np.reshape(ass, (101, 8*150, 4096))
print(ass.shape)

In [ ]:
if loss == "uniform":
    measure_batch_assignment_consistency(icd_assignment_list, loss="UniformCD")
else:
    measure_batch_assignment_consistency(ass, loss=loss)


In [ ]:
# plot consistency metrics

loss_funcs = ["infocd", "chamfer", "emd", "UniformCD"]
unique_list, consistency_list, change_list = [], [], []
for loss_func in loss_funcs:
    with open("data/" + loss_func + "_consistency.pkl", "rb") as f:
        consistency, unique, change = pickle.load(f)
        consistency_list.append(consistency)
        unique_list.append(unique)
        change_list.append(change)
print(change_list[0][-1], change_list[1][-1])

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 5))
plt.rcParams.update({'font.size': 12})
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14) 

limit= 50
loss_funcs = ["InfoCD", "CD", "EMD", "UniformCD"]

plot_dists(axes[0], consistency_list, loss_funcs, title="Backward consistency", 
           ylabel="no. of points", xlabel="Iterations", log=False, limit=limit, legend=False)
plot_dists(axes[1], unique_list, loss_funcs, title="Point coverage", 
           ylabel="", xlabel="Iterations", log=False, limit=limit, legend=False)
plot_dists(axes[2], change_list, loss_funcs, title="Correspondence variation", 
           ylabel="", xlabel="Iterations", log=False, limit=limit)


In [ ]:
# compare correspondences
#TODO: include consistency in top5 matches?
loss_func = "emd"
with open("data/assignments_" + "emd" + ".pkl", "rb") as f:
    emd_assignment = pickle.load(f)

with open("data/assignments_" + loss_func + ".pkl", "rb") as f:
    assignment = pickle.load(f)
    
for i, ass in enumerate(assignment):
    #measure_assignment_consistency(ass, emd_assignment[i][0])
    measure_assignment_consistency(ass)


#### Point-wise correspondence distance visualisation


In [ ]:
# load clouds
tgt_path = "data/target.pcd"
l1_path = "data/l1d.pcd"
l2_path = "data/l2d.pcd"
l3_path = "data/l3d.pcd"
l4_path = "data/l4d.pcd"

tgt = np.array(o3d.io.read_point_cloud(tgt_path).points)
l1 = np.array(o3d.io.read_point_cloud(l1_path).points)
l2 = np.array(o3d.io.read_point_cloud(l2_path).points)
l3 = np.array(o3d.io.read_point_cloud(l3_path).points)
l4 = np.array(o3d.io.read_point_cloud(l4_path).points)


In [ ]:
%autoreload 2

def get_point_weights(tgt, l1, l2, l3, l4, loss_func="chamfer"):
    w1, loss1 = get_point_distance(tgt, l1, loss_func)
    w2, loss2 = get_point_distance(tgt, l2, loss_func)
    w3, loss3 = get_point_distance(tgt, l3, loss_func)
    w4, loss4 = get_point_distance(tgt, l4, loss_func)

    high1, low1 = np.max(w1), np.min(w1)
    high2, low2 = np.max(w2), np.min(w2)
    high3, low3 = np.max(w3), np.min(w3)
    high4, low4 = np.max(w4), np.min(w4)

    high = max(high1, high2, high3, high4)
    low = min(low1, low2, low3, low4)
    
    losses = [loss1, loss2, loss3, loss4]
    print("loss1", loss1, "loss2", loss2, "loss3", loss3, "loss4", loss4)
    
    return high, low, w1, w2, w3, w4, losses


cd_high, cd_low, cd_w1, cd_w2, cd_w3, cd_w4, cd_losses = get_point_weights(tgt, l1, l2, l3, l4, loss_func="uniform")
uniform_high, uniform_low, uniform_w1, uniform_w2, uniform_w3, uniform_w4, uniform_losses = get_point_weights(tgt, l1, l2, l3, l4, loss_func="uniform")

high, low = max(cd_high, uniform_high), min(cd_low, uniform_low)

#colours = visualise_point_loss(uniform_w1, high, low, 'plasma_r')
#colours = visualise_point_loss(uniform_w4, uniform_high, uniform_low, 'plasma_r')
colours = visualise_point_loss(cd_w2, cd_high, cd_low, 'plasma_r')

point_cloud = o3d.geometry.PointCloud()
#print(colours, cloud.shape)
point_cloud.points = o3d.utility.Vector3dVector(l1)
point_cloud.colors = o3d.utility.Vector3dVector(colours)
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
point_cloud = o3d.geometry.PointCloud()
#print(colours, cloud.shape)
point_cloud.points = o3d.utility.Vector3dVector(tgt)
point_cloud.paint_uniform_color([0.1, 0.8, 0.6])

o3d.visualization.draw_geometries([point_cloud])

In [ ]:

print(normalize([uniform_losses]))
X = np.arange(4)

fig = plt.figure()

ax = fig.add_axes([0,0,1,1])
#ax.bar(X + 0.00, normalize([uniform_losses])[0], color = 'b', width = 0.25)
ax.bar(X + 0.25, normalize([cd_losses])[0], color = 'mediumslateblue', width = 0.8)
ax.set(ylabel="Loss (normalised)")
ax.set(title="Chamfer distance")

x1,x2,y1,y2 = plt.axis()  
plt.axis((x1,x2,0,0.7))
ax.set_xticks([])

In [ ]:
X = np.arange(4)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

plt.rcParams.update({'font.size': 22})
plt.rc('xtick', labelsize=10) 
#ax.bar(X + 0.00, normalize([uniform_losses])[0], color = 'b', width = 0.25)
ax.bar(X + 0.25, normalize([uniform_losses])[0], color = 'indianred', width = 0.8)
ax.set(ylabel="Loss (normalised)")
ax.set(title="UniformCD distance")
x1,x2,y1,y2 = plt.axis()  
plt.axis((x1,x2,0,0.7))
ax.set_xticks([])

#### CD EMD scatterplot


In [ ]:
# generate scatterplot to show correlations between EMD and Chamfer metrics for completion testset. CD, EMD values must saved as two lists in the 'out_metrics_<loss>.pkl file.
dataset = "mvp"

if dataset == "mvp":
    chamfer_path = "Density_aware_Chamfer_Distance/out_metrics_cd.pkl" #MVP 
    uniform_path = "Density_aware_Chamfer_Distance/out_metrics_uniform.pkl" #MVP 
else:
    chamfer_path = "data/out_metrics_cd.pkl" #PCN
    uniform_path = "data/out_metrics.pkl" #PCN

with open (chamfer_path, "rb") as f:
    chamfer_cd, chamfer_emd = pickle.load(f)
    
if dataset == "mvp":
    
    chamfer_cd, chamfer_emd = [x.detach().cpu().numpy() for x in chamfer_cd], [x.detach().cpu().numpy() for x in chamfer_emd]
    chamfer_cd, chamfer_emd = np.array(chamfer_cd).flatten(), np.array(chamfer_emd).flatten()
    chamfer_cd = chamfer_cd*1000
    chamfer_emd = chamfer_emd*100
else:
    chamfer_cd = np.array(chamfer_cd)
    chamfer_emd = np.array(chamfer_emd)*100

with open (uniform_path, "rb") as f:
    uniform_cd, uniform_emd = pickle.load(f)
    
if dataset == "mvp":
    uniform_cd, uniform_emd = [x.detach().cpu().numpy() for x in uniform_cd], [x.detach().cpu().numpy() for x in uniform_emd]
    uniform_cd, uniform_emd = np.array(uniform_cd).flatten(), np.array(uniform_emd).flatten()
    uniform_cd = uniform_cd*1000
    uniform_emd = uniform_emd*100
else:
    uniform_cd = np.array(uniform_cd)
    uniform_emd = np.array(uniform_emd)*100
    
    
print(np.average(uniform_cd), np.average(chamfer_cd), np.average(uniform_emd), np.average(chamfer_emd))


In [ ]:

plt.rcParams['figure.figsize']=(10,6)
plt.xlabel('CD (x$10^4$)', fontsize=20)
plt.ylabel('EMD (x$10^2$)', fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.title('MVP Testset Correlation between CD and EMD, VRC model', fontsize=20)

plt.scatter(uniform_cd, uniform_emd, s=3, color="lightseagreen", label="UniformCD loss")
plt.scatter(chamfer_cd, chamfer_emd, s=3, color="palevioletred", label="CD loss")
plt.legend(prop = { "size": 18 })

plt.show()

In [ ]:
### find correlation between chamfer and EMD

print("uniform r", scipy.stats.pearsonr(uniform_cd, uniform_emd))
print("chamfer r", scipy.stats.pearsonr(chamfer_cd, chamfer_emd))


#### Visualise loss curves for training VCN models


In [ ]:
# open train log and get eval records
train_log_file = "../experiments/Density_aware_Chamfer_Distance/log/vrcnet_plus_cd_debug_2024-03-11T17:00:03/train - Copy.log"
def get_metrics_from_log(train_log_file, uniform=False):
    with open(train_log_file, "rb") as f:
        train_log = f.readlines()
        
    eval_lines = []
    for line in train_log:
        if "curr" in str(line):
            eval_lines.append(line)
            
            
    # get results
    dcd, cd, emd, bcd = [], [] ,[], []

    for line in eval_lines:
        line = str(line)
        dcd.append(float(line.split("dcd: ")[1].split(";")[0]))
        cd.append(float(line.split("cd_t: ")[1].split(";")[0]))
        emd.append(float(line.split("emd: ")[1].split(";")[0]))
        if uniform:
            bcd.append(float(line.split("dcd: ")[1].split(";")[0]))

    
    return dcd, cd, emd, bcd


In [ ]:
train_log_file = "../experiments/Density_aware_Chamfer_Distance/log/vrcnet_plus_cd_debug_2024-03-11T17:00:03/train - Copy.log"
dcd_cd, cd_cd, emd_cd, _ = get_metrics_from_log(train_log_file)

train_log_file = "../experiments/Density_aware_Chamfer_Distance/log/vrcnet_plus_cd_debug_2024-03-11T17:00:03/train - Copy.log"
dcd_u, cd_u, emd_u, bcd_u = get_metrics_from_log(train_log_file)


In [ ]:
# visualise losses
iterations = list(range(2, len(cd_cd)*2+2, 2))

# Plotting the line graph
#plt.plot(iterations, dcd_cd, label='DCD')
#plt.plot(iterations, emd_cd, label='EMD')
plt.plot(iterations, cd_cd, label='CD')

# Adding labels and title
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss Evaluation')

# Displaying the graph
plt.legend()
plt.show()



